In [2]:
import os
from configparser import ConfigParser

config = ConfigParser()
config.read('../conf.ini')
openai_token=config['secret']['openai_token']
os.environ['OPENAI_API_KEY'] = openai_token

# Chains
- 좀 더 복잡한 Application을 구동하기 위해서 사용
    - 구성 요소를 체인으로 함께 구성
    - Application을 디버스, 유지 관리가 훨씬 편해짐

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["company", "product"],
    template="Can you recommend 5 good names for {company} that makes {product}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run({
    'company': "ABC Startup",
    'product': "colorful socks"
    }))



1. Joyful Socks
2. Colorful Crew
3. Rainbow Hues
4. Bright Feet
5. Happy Toes


## How to
### Async API
- asyncio 라이브러리를 통한 체인의 비동기 지원

In [6]:
import asyncio
import time

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def generate_serially():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    for _ in range(5):
        resp = chain.run(product="baseball equipment")
        print(resp)


async def async_generate(chain):
    resp = await chain.arun(product="baseball equipment")
    print(resp)


async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    tasks = [async_generate(chain) for _ in range(5)]
    await asyncio.gather(*tasks)


s = time.perf_counter()
await generate_concurrently()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Async function executed in {elapsed:0.2f} seconds." + "\033[0m")

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")



Diamond Sports Gear.


Diamond Glove Sports.


Hit & Field Supplies.


Batty's Baseball Supplies


Batter's Edge.
Async function executed in 0.84 seconds.


Diamond Sports.


Diamond Sports Gear.


Batters and Blades.


Diamond Bat Company.


PowerPlay Baseball Gear.
Serial executed in 2.72 seconds.
